In [1]:
import os
import openai
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import chroma, Pinecone
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from IPython.display import display, Markdown
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


_ = load_dotenv(find_dotenv()) # read local .env file
#openai key
openai.api_key = os.environ['OPENAI_API_KEY']
openai.model_name = os.environ['FAST_LLM_MODEL']
print(openai.model_name)

gpt-3.5-turbo-0613


c:\Users\test\anaconda3\envs\openai\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 加载PDF并做切片

In [2]:
loader = PyPDFLoader("../data/8B3608368D6AA693562457E20559FE6C.pdf")

In [3]:
data = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [5]:
embeddings = OpenAIEmbeddings()

## 用pinecone来做向量存储：

In [6]:
#init pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENV'])
index_name = "pdftest"
pinecone.create_index("pdftest2", dimension=1536, metric="euclidean")
#pinecone.list_indexes()

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=UTF-8', 'date': 'Tue, 04 Jul 2023 05:59:33 GMT', 'x-envoy-upstream-service-time': '1575', 'content-length': '131', 'server': 'envoy'})
HTTP response body: The index exceeds the project quota of 1 pods by 1 pods. Upgrade your account or change the project settings to increase the quota.


In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [ ]:
query = "请找出这家公司的生产的每一种设备，一一罗列出来"
docs = docsearch.similarity_search(query)
print(docs[0])

### 本地向量存储：

In [7]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator

In [8]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [9]:
template = ChatPromptTemplate.from_template(
    "This is a listing prospectus, suppose you a Stock investor,\
    Base on the information the information below:\
    {docs}, you need to find out the detial information \
     to answer the quetion of {query}\
     then output the anser in Chinese. NOte that, Do not summary \
     the information you found , but list the detail of them. \
     finally, output the answer in Chinese."
)
print(template)

input_variables=['docs', 'query'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs', 'query'], output_parser=None, partial_variables={}, template='This is a listing prospectus, suppose you a Stock investor,    Base on the information the information below:    {docs}, you need to find out the detial information      to answer the quetion of {query}     then output the anser in Chinese. NOte that, Do not summary      the information you found , but list the detail of them.      finally, output the answer in Chinese.', template_format='f-string', validate_template=True), additional_kwargs={})]


In [10]:
llm = OpenAI(model_name="gpt-3.5-turbo-0613",temperature=0.0)
chain = load_qa_chain(llm, chain_type="stuff")

c:\Users\test\anaconda3\envs\openai\lib\site-packages\langchain\llms\openai.py:189: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\test\anaconda3\envs\openai\lib\site-packages\langchain\llms\openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [11]:
response = chain.run(input_documents=docs, question=query, template = template)
display(Markdown(response))

NameError: name 'docs' is not defined